In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.ticker import ScalarFormatter, MultipleLocator
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import zscore

In [2]:
data_route = "C:\\workspace\\project_final\\data\\dataset_sample_10percent.csv"
data = pd.read_csv(data_route, encoding='utf-8')

In [3]:
# Z-score 기준 이상치 여부를 판단하는 임계값
z_threshold = 3

# 결과를 담을 리스트
outliers_data = []

# Z-score 계산 및 이상치 여부 확인 함수ㅊ
def calculate_z_scores(df, group_column):
    # 그룹 내에서 가격에 대한 Z-score 계산
    df[f'{group_column}_zscore'] = df.groupby(group_column)['price'].transform(lambda x: zscore(x, ddof=0))
    # Z-score가 임계값 이상인 경우 이상치로 간주
    df[f'{group_column}_outlier'] = df[f'{group_column}_zscore'].abs() > z_threshold
    return df


data['brandclass_name'] = data['class_name'] + '_' + data['brand_name']
# data['brandcate2_name'] = data['cate2_nm'] + '_' + data['brand_name']

# 기준 컬럼별로 Z-score 계산 및 이상치 판단
data = calculate_z_scores(data, 'supplier_code')
data = calculate_z_scores(data, 'brandclass_name')
data = calculate_z_scores(data, 'cate2_nm')

In [4]:
data['overall_outlier'] = data['supplier_code_outlier'] & data['brandclass_name_outlier'] & data['cate2_nm_outlier']

In [12]:
outliers = data[data['overall_outlier'] == True]

# 필요한 컬럼만 선택해서 출력
display(outliers[['prd_id', 'supplier_code', 'cate2_nm', 'class_name', 'brand_name', 
                  'supplier_code_zscore', 'brandclass_name_zscore', 'cate2_nm_zscore',
                  'overall_outlier', 'price']])

print(len(outliers)/len(data)*100, "% outlier based on Z-score")

,prd_id,supplier_code,cate2_nm,class_name,brand_name,supplier_code_zscore,brandclass_name_zscore,cate2_nm_zscore,overall_outlier,price
4206,1037438760,1020872,DIY/반제품가구,공간박스,오너클랜,15.359703,4.538480,5.636001,True,1435400
5344,93430532,1034195,책장/책꽂이,책장,베이직가구,3.486194,3.139458,19.859343,True,13490000
5547,1036855405,1034195,드레스룸/행거,이불장/옷장,베이직가구,8.297176,3.782171,29.769558,True,27350000
6667,50443268,1034195,식탁,식탁,베이직가구,3.548675,3.071473,11.492056,True,13670000
7331,1016357855,1043876,테이블,식탁세트,벤티스가구,4.202326,3.294012,5.331418,True,4041000
...,...,...,...,...,...,...,...,...,...,...
1588445,27808695,1031568,휴대폰케이스,핸드폰 액세서리,쇼베뉴엘,3.218066,3.211342,11.486311,True,116600
1589049,95070697,1036305,휴대폰케이스,핸드폰 액세서리,VERAX,3.193751,3.217321,3.344037,True,44300
1589891,1060780773,1040823,휴대폰케이스,핸드폰 액세서리,테일즈,3.087230,3.087230,3.073754,True,41900
1590011,1017439948,1036305,휴대폰케이스,핸드폰 액세서리,VERAX,3.967197,3.995334,4.087315,True,50900


0.15931739782421184 % outlier based on Z-score


In [14]:
# 기준 컬럼별로 Z-score 계산 및 이상치 판단
data = calculate_z_scores(data, 'brand_name')
data = calculate_z_scores(data, 'class_name')

data['overall_outlier'] = data['supplier_code_outlier'] & data['brandclass_name_outlier'] & data['cate2_nm_outlier'] & data['brand_name_outlier'] & data['class_name_outlier']

In [15]:
outliers = data[data['overall_outlier'] == True]

# 필요한 컬럼만 선택해서 출력
display(outliers[['prd_id', 'supplier_code', 'cate2_nm', 'class_name', 'brand_name', 
                  'supplier_code_zscore', 'brandclass_name_zscore', 'cate2_nm_zscore',
                  'overall_outlier', 'price']])

print(len(outliers)/len(data)*100, "% outlier based on Z-score")

,prd_id,supplier_code,cate2_nm,class_name,brand_name,supplier_code_zscore,brandclass_name_zscore,cate2_nm_zscore,overall_outlier,price
4206,1037438760,1020872,DIY/반제품가구,공간박스,오너클랜,15.359703,4.538480,5.636001,True,1435400
5344,93430532,1034195,책장/책꽂이,책장,베이직가구,3.486194,3.139458,19.859343,True,13490000
5547,1036855405,1034195,드레스룸/행거,이불장/옷장,베이직가구,8.297176,3.782171,29.769558,True,27350000
6667,50443268,1034195,식탁,식탁,베이직가구,3.548675,3.071473,11.492056,True,13670000
12302,67437053,1007899,소파,소파,자코모,9.340751,9.340751,7.588542,True,17300000
...,...,...,...,...,...,...,...,...,...,...
1575852,1050871082,1038059,기타휴대폰액세서리,음향/영상액세서리 기타,오너클랜,13.975888,8.013578,15.699262,True,1278300
1584683,1049563222,1038059,기타휴대폰액세서리,물병,오너클랜,7.654144,9.777158,8.603504,True,712900
1586137,1049524874,1038059,기타휴대폰액세서리,물병,오너클랜,3.937575,4.999315,4.431890,True,380500
1588445,27808695,1031568,휴대폰케이스,핸드폰 액세서리,쇼베뉴엘,3.218066,3.211342,11.486311,True,116600


0.10397093588291428 % outlier based on Z-score
